# Maquina de soporte vectorial

In [4]:
# librerías auxiliares
import numpy as np
import pandas as pd
from itertools import *

# modulos de sklearn
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.utils.class_weight import compute_class_weight

# metricas de desempeño
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

Importamos los datasets para cada modelo

In [2]:
data_estimador1 = pd.read_csv("dataset_clasificador_binario.csv")
data_estimador2 = pd.read_csv("dataset_clasificador_multiclase.csv")

## Construcción del modelo para identificar si habrá un fallo en las maquinas

El dataset que se usará para la construcción del modelo es el siguiente:

In [3]:
data_estimador1

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,H,L,M
0,-0.951417,-0.946356,0.067484,0.283054,-1.695647,0,0.0,0.0,1.0
1,-0.901428,-0.878954,-0.729604,0.634238,-1.648511,0,0.0,1.0,0.0
2,-0.951417,-1.013759,-0.227940,0.945286,-1.617087,0,0.0,1.0,0.0
3,-0.901428,-0.946356,-0.590253,-0.048061,-1.585664,0,0.0,1.0,0.0
4,-0.901428,-0.878954,-0.729604,0.002108,-1.554240,0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
9968,-0.601495,-1.081162,0.362908,-1.051442,-1.475680,0,0.0,0.0,1.0
9969,-0.551506,-1.081162,0.518981,-0.820665,-1.428545,0,1.0,0.0,0.0
9970,-0.501517,-0.946356,0.591443,-0.660124,-1.349985,0,0.0,0.0,1.0
9971,-0.501517,-0.878954,-0.729604,0.854982,-1.302849,0,1.0,0.0,0.0


In [39]:
def binary_svc(x, y, kernels, gammas,params_reg):
    """función que realizar experimentos sobre un SVM para clasificación
    
    x: numpy.Array, con las caracteristicas del problema
    y: numpy.Array, con la variable objetivo
    kernels: List[str], lista con valores a pasar 
        a sklearn correspondiente al kernel de la SVM
    gammas: List[float], lista con los valores a pasar a
        sklean correspondiente el valor de los coeficientes para usar en el
        kernel
    params_reg: List[float], lista con los valores a a pasar a 
        sklearn para ser usados como parametro de regularización

    retorna: pd.Dataframe con las siguientes columnas:
        - 3 columnas con los cuatro parametros: kernel, gamma, param de regularizacion
        - recall en el cojunto entrenamiento (promedio de los 5 folds)
        - recall en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del recall promedio en el cojunto test 
            (desviacion estandar de los 5 folds)

        - auc roc en el cojunto entrenamiento (promedio de los 5 folds)
        - auc roc en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del auc roc promedio en el cojunto test 
            (desviacion estandar de los 5 folds)

        - f1 score en el cojunto entrenamiento (promedio de los 5 folds)
        - f1 score en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del f1 score promedio en el cojunto test 
            (desviacion estandar de los 5 folds)
        - % de Vectores de Soporte promedio para los 5 folds (0 a 100)
    """
    idx = 0
    kf = StratifiedKFold(n_splits=5)
    # crear una lista con la combinaciones de los elementos de cada lisa de hiperparametros
    kernels_gammas_regs = list(product(kernels, gammas, params_reg))
    resultados = pd.DataFrame()
    
    for params in kernels_gammas_regs:
        kernel, gamma, param_reg = params
        print("parametros usados", params) # puede usar para ver los params
        errores_train_recall = []
        errores_test_recall = []

        errores_train_auc = []
        errores_test_auc = []

        pct_support_vectors = []
        
        for train_index, test_index in kf.split(x,y):
            X_train, X_test = x[train_index], x[test_index]
            y_train, y_test = y[train_index], y[test_index]  

            # calculamos los pesos para cada clase
            class_weight = compute_class_weight(class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train )
          
           
            # Balancear los datos
            nm = RandomUnderSampler(sampling_strategy=0.5)
            X_train, y_train = nm.fit_resample(X_train, y_train)
          
            # definir el modelo
            svm = SVC(kernel = kernel, gamma = gamma, C = param_reg, probability=True, class_weight = {0:class_weight[0], 1:class_weight[1]})             
            
            # Entrenar el modelo
            svm.fit(X=X_train, y=y_train)
            # prediccion del modelo
            y_train_pred = svm.predict(X=X_train)
            y_test_pred = svm.predict(X=X_test)
            
            # calculo de errores y pct de vectores de soporte
            # calculo del recall
            errores_train_recall.append(recall_score(y_true = y_train, y_pred = y_train_pred))
            errores_test_recall.append(recall_score(y_true = y_test, y_pred = y_test_pred))
            # calculo del area bajo la curva roc
            errores_train_auc.append(roc_auc_score(y_true=y_train, y_score=svm.predict_proba(X_train)[:, 1]))
            errores_test_auc.append(roc_auc_score(y_true=y_test, y_score=svm.predict_proba(X_test)[:, 1]))

            # contar muestras de entrenamiento
            n_train = X_train.shape[0]
            
            # solo tenemos una SVM
            pct_vs = (len(svm.support_)/n_train)*100
            pct_support_vectors.append(pct_vs)
        
        resultados.loc[idx,'kernel'] = kernel
        resultados.loc[idx,'gamma'] = gamma
        resultados.loc[idx,'param_reg'] = param_reg

        resultados.loc[idx,'error de entrenamiento (recall)'] = np.mean(errores_train_recall)
        resultados.loc[idx,'error de prueba (recall)'] = np.mean(errores_test_recall)
        resultados.loc[idx,'error de prueba (intervalo de confianza - recall)'] = np.std(errores_test_recall)

        resultados.loc[idx,'error de entrenamiento (auc)'] = np.mean(errores_train_auc)
        resultados.loc[idx,'error de prueba (auc)'] = np.mean(errores_test_auc)
        resultados.loc[idx,'error de prueba (intervalo de confianza - auc)'] = np.std(errores_test_auc)

        resultados.loc[idx,'% de vectores de soporte'] = np.mean(pct_support_vectors)
        idx+=1
    return (resultados)

Realizamos los experimentos para encontrar la mejor combinación de hiperparametros

In [40]:
# vamos a realizar los experimentos
x_estimador1 = data_estimador1.drop(['Target'], axis=1).values
y_estimador1 = y_estimator1 = data_estimador1['Target'].values
resultadosSVC = binary_svc(x = x_estimador1,y=y_estimador1,
                                 kernels=['linear', 'rbf'],
                                 gammas = [0.01,0.1],
                                 params_reg = [0.001, 0.01,0.1,10])

parametros usados ('linear', 0.01, 0.001)
parametros usados ('linear', 0.01, 0.01)
parametros usados ('linear', 0.01, 0.1)
parametros usados ('linear', 0.01, 10)
parametros usados ('linear', 0.1, 0.001)
parametros usados ('linear', 0.1, 0.01)
parametros usados ('linear', 0.1, 0.1)
parametros usados ('linear', 0.1, 10)
parametros usados ('rbf', 0.01, 0.001)
parametros usados ('rbf', 0.01, 0.01)
parametros usados ('rbf', 0.01, 0.1)
parametros usados ('rbf', 0.01, 10)
parametros usados ('rbf', 0.1, 0.001)
parametros usados ('rbf', 0.1, 0.01)
parametros usados ('rbf', 0.1, 0.1)
parametros usados ('rbf', 0.1, 10)


ver los 5 mejores modelos

In [41]:
resultadosSVC.sort_values('error de prueba (recall)', ascending=False).head(5)

,kernel,gamma,param_reg,error de entrenamiento (recall),error de prueba (recall),error de prueba (intervalo de confianza - recall),error de entrenamiento (auc),error de prueba (auc),error de prueba (intervalo de confianza - auc),% de vectores de soporte
0,linear,0.01,0.001,1.0,1.0,0.0,0.907429,0.893931,0.061288,69.595960
1,linear,0.01,0.010,1.0,1.0,0.0,0.901400,0.892518,0.060968,69.368687
4,linear,0.10,0.001,1.0,1.0,0.0,0.904572,0.889622,0.059739,69.520202
5,linear,0.10,0.010,1.0,1.0,0.0,0.905020,0.892787,0.060081,69.444444
8,rbf,0.01,0.001,1.0,1.0,0.0,0.943019,0.917692,0.045991,71.464646


## Construcción del modelo para identificar el tipo de fallo

El dataset que se usará para la construcción del modelo es el siguiente:

In [ ]:
data_estimador2

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type,H,L,M
0,-0.551506,-0.609342,7.369473,-3.549861,0.551157,2.0,0.0,1.0,0.0
1,-0.551506,-0.676745,-0.718456,2.580797,1.305329,2.0,0.0,1.0,0.0
2,-0.601495,-0.744148,-0.467624,0.132547,1.572432,3.0,0.0,1.0,0.0
3,-0.801450,-1.215968,-1.431932,2.079106,1.698127,1.0,0.0,1.0,0.0
4,-0.851439,-1.283370,-0.707308,1.236266,1.729551,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
325,-0.701473,-0.137523,4.080791,-2.385939,1.729551,3.0,0.0,1.0,0.0
326,-0.751461,-0.339731,-1.365044,2.681135,-1.507104,2.0,0.0,1.0,0.0
327,-0.751461,-0.407134,-0.997157,2.099174,1.242481,1.0,0.0,1.0,0.0
328,-0.851439,-0.474537,-1.125360,1.617551,1.541008,1.0,0.0,1.0,0.0


In [46]:
def multiclass_svc(x, y, kernels, gammas,params_reg, estrategia = 'ovr'):
    """función que realizar experimentos sobre un SVM para clasificación multi clase
    
    x: numpy.Array, con las caracteristicas del problema
    y: numpy.Array, con la variable objetivo
    kernels: List[str], lista con valores a pasar 
        a sklearn correspondiente al kernel de la SVM
    gammas: List[float], lista con los valores a pasar a
        sklean correspondiente el valor de los coeficientes para usar en el
        kernel
    params_reg: List[float], lista con los valores a a pasar a 
        sklearn para ser usados como parametro de regularización
    estrategia: str, valor configurado por defecto para indicar que se usará ovr 
        (para one vs rest)
    
    retorna: pd.Dataframe con las siguientes columnas:
        - 3 columnas con los tres parametros: kernel, gamma, param de regularizacion
        - recall en el cojunto entrenamiento (promedio de los 5 folds)
        - recall en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del recall promedio en el cojunto test 
            (desviacion estandar de los 5 folds)

        - auc roc en el cojunto entrenamiento (promedio de los 5 folds)
        - auc roc en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del auc roc promedio en el cojunto test 
            (desviacion estandar de los 5 folds)

        - f1 score en el cojunto entrenamiento (promedio de los 5 folds)
        - f1 score en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del f1 score promedio en el cojunto test 
            (desviacion estandar de los 5 folds)
        - % de Vectores de Soporte promedio para los 5 folds (0 a 100)
    """
    idx = 0
    kf = StratifiedKFold(n_splits=5)
    # crear una lista con la combinaciones de los elementos de cada list
    kernels_gammas_regs = list(product(kernels, gammas, params_reg))
    resultados = pd.DataFrame()
    
    for params in kernels_gammas_regs:
        kernel, gamma, param_reg = params
        print("parametros usados", params) # puede usar para ver los params
        errores_train_recall = []
        errores_test_recall = []

        errores_train_auc = []
        errores_test_auc = []

        pct_support_vectors = []
        
        for train_index, test_index in kf.split(x,y):
            X_train, X_test = x[train_index], x[test_index]
            y_train, y_test = y[train_index], y[test_index]  

            # Balancear los datos
            smote = SMOTE(k_neighbors=3) 
            X_train, y_train = smote.fit_resample(X_train, y_train)

            # definir el modelo
            svm = SVC(kernel = kernel, gamma = gamma, C = param_reg, probability=True)
            
            # se usará la estrategia ovr
            svm = OneVsRestClassifier(svm)               
            
            # Entrenar el modelo
            svm.fit(X=X_train, y=y_train)
            # prediccion del modelo
            y_train_pred = svm.predict(X=X_train)
            y_test_pred = svm.predict(X=X_test)
            # error y pct de vectores de soporte
            # calculo del recall
            errores_train_recall.append(recall_score(y_true = y_train, y_pred = y_train_pred, average='weighted'))
            errores_test_recall.append(recall_score(y_true = y_test, y_pred = y_test_pred, average='weighted'))
            # calculo del area bajo la curva roc
            errores_train_auc.append(roc_auc_score(y_true=y_train, y_score=svm.predict_proba(X_train), multi_class='ovr'))
            errores_test_auc.append(roc_auc_score(y_true=y_test, y_score=svm.predict_proba(X_test), multi_class='ovr'))

            # contar muestras de entrenamiento
            n_train = X_train.shape[0]
            # en esta estrategia se realizar una SVM por cada clase
            # por lo tanto tenemos que acceder a cada una de la SVM
            # lee la documentacion
            num_vs = np.mean([len(svc.support_) for svc in svm.estimators_])
            pct_vs = (num_vs/n_train)*100
            pct_support_vectors.append(pct_vs)
        
        resultados.loc[idx,'kernel'] = kernel
        resultados.loc[idx,'gamma'] = gamma
        resultados.loc[idx,'param_reg'] = param_reg
        resultados.loc[idx,'estrategia'] = estrategia

        resultados.loc[idx,'error de entrenamiento (recall)'] = np.mean(errores_train_recall)
        resultados.loc[idx,'error de prueba (recall)'] = np.mean(errores_test_recall)
        resultados.loc[idx,'error de prueba (intervalo de confianza - recall)'] = np.std(errores_test_recall)

        resultados.loc[idx,'error de entrenamiento (auc)'] = np.mean(errores_train_auc)
        resultados.loc[idx,'error de prueba (auc)'] = np.mean(errores_test_auc)
        resultados.loc[idx,'error de prueba (intervalo de confianza - auc)'] = np.std(errores_test_auc)

        resultados.loc[idx,'% de vectores de soporte'] = np.mean(pct_support_vectors)
        idx+=1
    return (resultados)

Realizamos los experimentos para encontrar la mejor combinación de hiperparametros

In [47]:
# vamos a realizar los experimentos
x_estimador2 = data_estimador2.drop(['Failure Type'], axis=1).values
y_estimador2 = data_estimador2['Failure Type'].values
resultadosSVC_ovr = multiclass_svc(x = x_estimador2,y=y_estimador2,
                                 kernels=['linear', 'rbf'],
                                 gammas = [0.01,0.1],
                                 params_reg = [0.001, 0.01,0.1,10])

parametros usados ('linear', 0.01, 0.001)
parametros usados ('linear', 0.01, 0.01)
parametros usados ('linear', 0.01, 0.1)
parametros usados ('linear', 0.01, 10)
parametros usados ('linear', 0.1, 0.001)
parametros usados ('linear', 0.1, 0.01)
parametros usados ('linear', 0.1, 0.1)
parametros usados ('linear', 0.1, 10)
parametros usados ('rbf', 0.01, 0.001)
parametros usados ('rbf', 0.01, 0.01)
parametros usados ('rbf', 0.01, 0.1)
parametros usados ('rbf', 0.01, 10)
parametros usados ('rbf', 0.1, 0.001)
parametros usados ('rbf', 0.1, 0.01)
parametros usados ('rbf', 0.1, 0.1)
parametros usados ('rbf', 0.1, 10)


ver los 5 mejores modelos

In [48]:
resultadosSVC_ovr.sort_values('error de prueba (auc)', ascending=False).head(5)

,kernel,gamma,param_reg,estrategia,error de entrenamiento (recall),error de prueba (recall),error de prueba (intervalo de confianza - recall),error de entrenamiento (auc),error de prueba (auc),error de prueba (intervalo de confianza - auc),% de vectores de soporte
3,linear,0.01,10.0,ovr,0.955356,0.915152,0.058760,0.997149,0.991282,0.008856,12.080368
7,linear,0.10,10.0,ovr,0.960924,0.906061,0.063852,0.997385,0.990653,0.009980,11.621567
2,linear,0.01,0.1,ovr,0.925194,0.851515,0.091111,0.994497,0.987359,0.014676,30.271848
11,rbf,0.01,10.0,ovr,0.950886,0.875758,0.084306,0.996410,0.987115,0.013790,23.198502
6,linear,0.10,0.1,ovr,0.924663,0.845455,0.101594,0.994113,0.985887,0.015414,30.344413
